In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import spectral
from sklearn import preprocessing
from sklearn.cluster import KMeans 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import metrics
#from xgboost import XGBClassifier
from sklearn.decomposition import PCA
#from sklearn.ensemble import GradientBoostingClassifier
#from sklearn.ensemble import AdaBoostClassifier
#import lightgbm as lgb
import joblib

In [ ]:
input_image = loadmat('F:\data\Indian_pines_corrected.mat')['indian_pines_corrected']
output_image = loadmat('F:\data\Indian_pines_gt.mat')['indian_pines_gt']

In [ ]:
input_image.shape

In [ ]:
output_image.shape

In [2]:
## 降维操作
def pca_hpi(input_image, dimension=30):
    spectral = list()
    for i in range(input_image.shape[0]):
        for j in range(input_image.shape[1]):
            spectral.append(input_image[i][j,])
    spectral_array = np.array(spectral)
    ## 降维
    pca = PCA(n_components=dimension,svd_solver='auto')
    pca.fit(spectral_array)
    pca_input_iamge = pca.transform(spectral_array)
    ## 姜维之后恢复 原来形状
    pca_return_array = np.vsplit(pca_input_iamge,145)
    pca_return_array = np.array(pca_return_array)
    print (pca.explained_variance_ratio_)
    return pca_return_array

In [ ]:
pca_return_array = pca_hpi(input_image,30)  # 降维后Cr成为30

In [ ]:
pca_return_array.shape

In [ ]:
pca_return_array[0].shape


## patch的生成   3D tensor

In [3]:
# 可以提取出高光谱数据的一个channel的数据 145*145
def split_one_channel(A,channel):
    row = A.shape[0]
    d = [0]*row
    for i in range(row):
        d[i] = A[i][:,channel-1]
    e = np.vstack((d[i] for i in range(row)))
    return e   
# 参数说明
# channel 通道
# A 高光谱立方块


# 依据patch大小（Filter 大小）对矩阵进行镜像处理
def mirror_matrix(A, fil):
    pad = int((fil-1)/2)
    for  i in range(1,pad+1):
        up = np.array(A[2*i-1,:])
        down = np.array(A[-2*i,:])
        Matrix_up_down = np.vstack([up,A,down])
        left = np.array(Matrix_up_down[:,2*i-1]).reshape(len(Matrix_up_down),1)
        right = np.array(Matrix_up_down[:,-2*i]).reshape(len(Matrix_up_down),1)
        Matrix_left_right = np.hstack([left,Matrix_up_down,right,])
        A = Matrix_left_right
    return A

# 参数说明
# n   array size
# fil  filter size5*5
# pad = (fil-1)/2  padding大小


def recreate_whole_data(dim,fil,input_image):
    whole_origin_matrix_array = np.array([split_one_channel(input_image,i) for i in range(1,dim+1)])
    whole_mirror_matrix_array = np.array([mirror_matrix(whole_origin_matrix_array[i],fil) for i in range(dim)])
    return whole_mirror_matrix_array


## 分割patch  work out 一个维度的patch
def create_patch_one(A,fil):
    whole_arr = list()
    row = A.shape[0]-fil+1
    column = A.shape[1]-fil+1
    for i in range(row):
        for j in range(column):
            temp_arr = list()
            for z in range(fil):
                p = A[i+z,j:j+fil]
                temp_arr.append(p)
            one_arr = np.vstack(temp_arr)
            whole_arr.append(one_arr)
    return np.array(whole_arr)
    
def create_patch_whole(A,fil,dim):
    temp = [create_patch_one(split_one_channel(A,i),fil) for i in range(1,dim+1)]
    return np.array(temp)


def create_patch_cube(patch,fil,dim):
    temp = list()
    nums = patch[0].shape[0] # the nums of every dim of windows  n - f + 1
    for j in range(nums):
        temp_re = np.vstack([patch[i][j] for i in range(dim)]).reshape(dim,fil,fil)
        temp_re1= temp_re.transpose(2,1,0) # switch 1,3 dim
        temp_re2 = temp_re1.transpose(1,0,2) # switch 1,2 dim
        temp.append(temp_re2)
    patch_cubes = np.array(temp)
    return patch_cubes
    
    

### 测试

In [ ]:
## 将每个 filxfil (3x3)的patcha取出来， 并将每个channel的信息放入一个集合
patch_dim = create_patch_whole(pca_return_array,3,30)

In [ ]:
patch_dim.shape  # 30 个dim  every dim 20499个window (sample)    window(3x3)

In [ ]:
143*143

In [ ]:
patch_cubes = create_patch_cube(patch_dim,3, 30)

In [ ]:
patch_cubes.shape  # 20499 samples   one sampe is a 3D tensor (3x3x30)

In [ ]:
## 测试成功

## 开始联合标签

In [ ]:
patch_cubes[0].shape

In [4]:
DIM = 30
FIL = 3

In [5]:
inp = loadmat('F:\data\Indian_pines_corrected.mat')['indian_pines_corrected']
out = loadmat('F:\data\Indian_pines_gt.mat')['indian_pines_gt']

In [ ]:
list1 = [out[i][j]for i in range(145)for j in range(145)]
set(list1)

In [6]:
## 降维
pca_return_array_alpha = pca_hpi(inp,DIM) 

[6.84937994e-01 2.35313708e-01 1.49635476e-02 8.21541515e-03
 6.95011265e-03 5.17010703e-03 3.99681431e-03 3.62359687e-03
 3.07127396e-03 2.93211894e-03 2.67352983e-03 2.49230109e-03
 2.24688362e-03 1.89388745e-03 1.69433605e-03 1.56043456e-03
 1.53162471e-03 1.35012974e-03 1.00138773e-03 9.24873537e-04
 8.47882716e-04 7.64371870e-04 6.64568247e-04 6.45636943e-04
 6.16268833e-04 5.61261295e-04 5.42934379e-04 5.15475231e-04
 4.20758532e-04 3.62389167e-04]


In [7]:
# padd 并镜像
pca_return_array_beta = recreate_whole_data(DIM,FIL,pca_return_array_alpha)


In [8]:
pca_return_array_beta.shape

(30, 147, 147)

In [ ]:
# pca_return_array_beta.transpose(2,1,0).shape

In [ ]:
# pca_return_array_beta.shape

In [9]:
## 将pca_return beta 变为 147--3  x  147--2  x 30 形状
pca_return_array_beta = pca_return_array_beta.transpose(2,1,0)

In [10]:
pca_return_array_beta.shape

(147, 147, 30)

In [11]:
pca_return_array_beta = pca_return_array_beta.transpose(1,0,2)

In [12]:
pca_return_array_beta.shape

(147, 147, 30)

In [13]:
## ## 将每个 filxfil (3x3)的patch取出来， 并将每个channel的信息放入一个集合
patch_dim = create_patch_whole(pca_return_array_beta,3,30)

In [14]:
patch_dim.shape  # 30 个dim  every dim 20735个window (sample)    window(3x3)

(30, 21025, 3, 3)

In [15]:
patch_cubes = create_patch_cube(patch_dim,3, 30)

In [16]:
patch_cubes.shape   # 21025个 sample 每个sample是一个patch 30dim  3x3

(21025, 3, 3, 30)

In [17]:
patch_cubes[0].transpose(2,1,0)

array([[[ 5.18403234e+03,  5.60138374e+03,  5.18403234e+03],
        [ 4.59543714e+03,  5.01490598e+03,  4.59543714e+03],
        [ 5.18403234e+03,  5.60138374e+03,  5.18403234e+03]],

       [[-7.79019533e+02, -2.02345009e+03, -7.79019533e+02],
        [ 2.36435179e+03,  1.45686326e+03,  2.36435179e+03],
        [-7.79019533e+02, -2.02345009e+03, -7.79019533e+02]],

       [[ 1.81723902e+02,  3.50134661e+02,  1.81723902e+02],
        [-2.98906147e+01,  7.26970489e+01, -2.98906147e+01],
        [ 1.81723902e+02,  3.50134661e+02,  1.81723902e+02]],

       [[-1.41688645e+02, -5.28465052e+02, -1.41688645e+02],
        [ 1.23180845e+02,  7.12049267e+01,  1.23180845e+02],
        [-1.41688645e+02, -5.28465052e+02, -1.41688645e+02]],

       [[-3.26279891e+02,  1.48088292e+02, -3.26279891e+02],
        [-4.15941999e+02, -4.35686988e+02, -4.15941999e+02],
        [-3.26279891e+02,  1.48088292e+02, -3.26279891e+02]],

       [[-4.86495937e+02, -2.88359036e+02, -4.86495937e+02],
        [ 1.56

In [18]:
patch_cubes[0].shape

(3, 3, 30)

In [19]:
out.shape

(145, 145)

In [20]:
out_labels = out.reshape(145*145,)

In [ ]:
out_labels

In [21]:
def useful_sample_lable(patch_cubes):
    sample_lable_list = list()
    for i in range(len(patch_cubes)):
        if out_labels[i] != 0:
            sample_lable_list.append((patch_cubes[i],out_labels[i]))
    sample_lable_array = np.array(sample_lable_list)
    return sample_lable_array

In [ ]:
sample_lable_array = useful_sample_lable(patch_cubes)

In [ ]:
sample_lable_array.shape 

In [ ]:
sample_lable_array[0][0].shape  # sample shape

In [ ]:
sample_lable_array[0][1].shape  # label shape

In [ ]:
sample_lable_array[0][1] # label 

## 搭建 2D-CNN

In [ ]:
import keras

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D,MaxPooling2D
from keras.optimizers import SGD
import numpy as np

## 30x5x5 变为 5x5x30  transpose(2,1,0)    transpose(1,0,2)

In [ ]:
model = Sequential()
# input : 5x5 patch with 30 channels - > (5,5,30)
# layer1 Conv2d  30 x 3 = 90 filters
model.add(Conv2D(90, (3, 3), activation='relu', input_shape=(100, 100, 3)))

model.add(Conv2D(270, (3, 3), activation='relu'))

          
model.add(Dense(180, activation='relu'))
model.add(Dense(16, activation='softmax'))
          
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)
          
# model.fit(x_train, y_train, batch_size=32, epochs=10)
# score = model.evaluate(x_test, y_test, batch_size=32)